In [15]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [16]:
import hashlib

def generate_document_id(doc):
    # combined = f"{doc['course']}-{doc['question']}"
    combined = f"{doc['course']}-{doc['question']}-{doc['text'][:10]}"
    hash_object = hashlib.md5(combined.encode())
    hash_hex = hash_object.hexdigest()
    document_id = hash_hex[:8]
    return document_id

In [17]:
for doc in documents:
    doc['id'] = generate_document_id(doc)

In [19]:
documents[4]

{'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
 'section': 'General course-related questions',
 'question': 'Course - What can I do before the course starts?',
 'course': 'data-engineering-zoomcamp',
 'id': '63394d91'}

In [5]:
from collections import defaultdict

In [6]:
hashes = defaultdict(list)

for doc in documents:
    doc_id = doc['id']
    hashes[doc_id].append(doc)

In [7]:
len(hashes), len(documents)

(947, 948)

In [8]:
for k, values in hashes.items():
    if len(values) > 1:
        print(k, len(values))

593f7569 2


In [12]:
len(hashes.keys())

947

In [13]:
hashes['593f7569']

[{'text': "They both do the same, it's just less typing from the script.\nAsked by Andrew Katoch, Added by Edidiong Esu",
  'section': '6. Decision Trees and Ensemble Learning',
  'question': 'Does it matter if we let the Python file create the server or if we run gunicorn directly?',
  'course': 'machine-learning-zoomcamp',
  'id': '593f7569'},
 {'text': "They both do the same, it's just less typing from the script.",
  'section': '6. Decision Trees and Ensemble Learning',
  'question': 'Does it matter if we let the Python file create the server or if we run gunicorn directly?',
  'course': 'machine-learning-zoomcamp',
  'id': '593f7569'}]

In [1]:
import json
import os

In [2]:
os.getcwd()

'/mnt/d/repos/llm-zoomcamp/03-vector-search/eval'

In [3]:

with open('documents-with-ids.json', 'wt') as f_out:
    json.dump(documents, f_out, indent=2)

In [3]:
!head -n 20 documents-with-ids.json

In [5]:
prompt_template = """
You emulate a student who's taking our course.
Formulate 5 questions this student might ask based on a FAQ record. The record
should contain the answer to the questions, and the questions should be complete and not too short.
If possible, use as fewer words as possible from the record. 

The record:

section: {section}
question: {question}
answer: {text}

Provide the output in parsable JSON without using code blocks:

["question1", "question2", ..., "question5"]
""".strip()

In [9]:
import os
from openai import OpenAI

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=OPENAI_API_KEY)

In [12]:
def generate_questions(doc):
    prompt = prompt_template.format(**doc)

    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )

    json_response = response.choices[0].message.content
    return json_response

In [13]:
from tqdm.auto import tqdm

In [21]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp',
 'id': 'c02e79ef'}

In [22]:
results = {}

In [23]:
for doc in tqdm(documents): 
    doc_id = doc['id']
    if doc_id in results:
        continue

    questions = generate_questions(doc)
    results[doc_id] = questions

  0%|          | 0/948 [00:00<?, ?it/s]

In [31]:
json.loads(results["c02e79ef"])

['What is the start date and time for the course?',
 'How can I keep track of the course schedule?',
 "When is the first 'Office Hours' session for the course?",
 'Is there a specific form to register before the course starts?',
 "Where can I join the course's Telegram channel for announcements?"]

In [39]:
with open('results.bin', 'wb') as f_out:
    pickle.dump(results, f_out)

In [24]:
import pickle

In [40]:
with open('results.bin', 'rb') as f_in:
    results = pickle.load(f_in)

In [41]:
results["c02e79ef"]

'[\n  "What is the start date and time for the course?",\n  "How can I keep track of the course schedule?",\n  "When is the first \'Office Hours\' session for the course?",\n  "Is there a specific form to register before the course starts?",\n  "Where can I join the course\'s Telegram channel for announcements?"\n]'

In [42]:
results['1f6520ca']

'[\n  "Where can I find the prerequisites for this course?",\n  "What should I check before enrolling in the course?",\n  "Is there a way to know the required background for this course?",\n  "How do I access the list of prerequisites for the course?",\n  "Where is the information about the course prerequisites located?"\n]'

In [47]:
parsed_results = {}

for doc_id, json_questions in results.items():
    parsed_results[doc_id] = json.loads(json_questions)

In [48]:
doc_index = {d['id']: d for d in documents}

In [50]:
documents[0]["course"]

'data-engineering-zoomcamp'

In [52]:
# doc_index

In [53]:
final_results = []

for doc_id, questions in parsed_results.items():
    course = doc_index[doc_id]['course']
    for q in questions:
        final_results.append((q, course, doc_id))

In [54]:
final_results

[('What is the start date and time for the course?',
  'data-engineering-zoomcamp',
  'c02e79ef'),
 ('How can I keep track of the course schedule?',
  'data-engineering-zoomcamp',
  'c02e79ef'),
 ("When is the first 'Office Hours' session for the course?",
  'data-engineering-zoomcamp',
  'c02e79ef'),
 ('Is there a specific form to register before the course starts?',
  'data-engineering-zoomcamp',
  'c02e79ef'),
 ("Where can I join the course's Telegram channel for announcements?",
  'data-engineering-zoomcamp',
  'c02e79ef'),
 ('Where can I find the prerequisites for this course?',
  'data-engineering-zoomcamp',
  '1f6520ca'),
 ('What should I check before enrolling in the course?',
  'data-engineering-zoomcamp',
  '1f6520ca'),
 ('Is there a way to know the required background for this course?',
  'data-engineering-zoomcamp',
  '1f6520ca'),
 ('How do I access the list of prerequisites for the course?',
  'data-engineering-zoomcamp',
  '1f6520ca'),
 ('Where is the information about th

In [55]:
import pandas as pd

In [56]:
df = pd.DataFrame(final_results, columns=['question', 'course', 'document'])

In [57]:
df.head()

,question,course,document
0,What is the start date and time for the course?,data-engineering-zoomcamp,c02e79ef
1,How can I keep track of the course schedule?,data-engineering-zoomcamp,c02e79ef
2,When is the first 'Office Hours' session for t...,data-engineering-zoomcamp,c02e79ef
3,Is there a specific form to register before th...,data-engineering-zoomcamp,c02e79ef
4,Where can I join the course's Telegram channel...,data-engineering-zoomcamp,c02e79ef


In [59]:
df.to_csv('ground-truth-data.csv', index=False)

In [60]:
!head -n 5 ground-truth-data.csv

question,course,document
What is the start date and time for the course?,data-engineering-zoomcamp,c02e79ef
How can I keep track of the course schedule?,data-engineering-zoomcamp,c02e79ef
When is the first 'Office Hours' session for the course?,data-engineering-zoomcamp,c02e79ef
Is there a specific form to register before the course starts?,data-engineering-zoomcamp,c02e79ef
